In [1]:
#here we only explore explore/exploit strats, not much care is given to rewards(information gathering and utilization)
import gym
import numpy as np
import tqdm as tqdm
env = gym.make("FrozenLake-v0")

In [2]:
#Q = np.zeros((env.observation_space.n, env.action_space.n)) 
#Q[0]

In [3]:
def pure_exploration_strat_fl(env, n_ep = 1):
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    V = np.zeros((env.observation_space.n)) 
    N = np.zeros((env.observation_space.n, env.action_space.n)) #used to prevent overflow
    theta = 1e-10
    
    gamma = 0.99
    name = 'Pure exploration'
    for e in range(n_ep):
        state = env.reset()
        while True:
            action = np.random.randint((env.action_space.n))
            #print(type(action))
            new_state, reward, done, _ = env.step(action)
            N[state][action] += 1
            Q[state][action] += (reward + gamma*V[new_state])/ N[state][action]
            
            #reason for subtraction is because in this env, rewards are 0 or 1. having a constant +ve reward may cause hangups 
            if done == True:
                break
            #print(np.max(np.abs(V - np.max(Q, axis = 1))))
            #if np.max(np.abs(V - np.max(Q, axis = 1))) < theta:
            #    break
            state = new_state
        V = np.max(Q, axis = 1)
            #Q_s[e] = Q
            #returns[e] = reward
            #actions[e] = action
    return V,Q

In [4]:
#pure_exploration_strat_fl(env, n_ep = 5000)

In [5]:
def pure_exploitation_strat_fl(env, n_ep = 1):
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    V = np.zeros((env.observation_space.n)) 
    N = np.zeros((env.observation_space.n, env.action_space.n)) #used to prevent overflow
    theta = 1e-10
    
    gamma = 0.99
    name = 'Pure exploitation'
    for e in range(n_ep):
        state = env.reset()
        while True:
            action = np.argmax(Q[state])
            new_state, reward, done, _ = env.step(action)
            N[state][action] += 1
            Q[state][action] += (reward + gamma*V[new_state])/ N[state][action]
            
            #reason for subtraction is because in this env, rewards are 0 or 1. having a constant +ve reward may cause hangups 
            if done == True:
                break
            #print(np.max(np.abs(V - np.max(Q, axis = 1))))
            #if np.max(np.abs(V - np.max(Q, axis = 1))) < theta:
            #    break
            state = new_state
        V = np.max(Q, axis = 1)
            #Q_s[e] = Q
            #returns[e] = reward
            #actions[e] = action
    return V,Q

In [6]:
#pure_exploitation_strat_fl(env, n_ep = 10)

In [7]:
def pure_exploitation_strat(env, n_ep = 300):
    Q = np.zeros((env.action_space.n)) #remember, mab has only one state so Q table will be 1 by no. of actions
    N = np.zeros((env.action_space.n)) #counts the number of times an action was selected(basically the discount factor)
    
    Q_s = np.empty((n_ep, env.action_space.n))
    returns = np.empty(n_ep)
    actions = np.empty(n_ep)
    
    name = 'Pure exploitation'
    for e in tqdm(range(n_ep)):
        action = np.argmax(Q)
        N[action]+= 1
        _, reward, _, _ = env.step(action)
        Q[action] += (reward - Q[action])/N[action] 
        #reason for subtraction is because in this env, rewards are 0 or 1. having a constant +ve reward may cause hangups 
        
        Q_s[e] = Q
        returns[e] = reward
        actions[e] = action
    return name, returns, Qe, actions

In [8]:
def pure_random_strat(env, n_ep = 300):
    Q = np.zeros((env.action_space.n)) #remember, mab has only one state so Q table will be 1 by no. of actions
    N = np.zeros((env.action_space.n)) #counts the number of times an action was selected(basically the discount factor)
    
    Q_s = np.empty((n_ep, env.action_space.n))
    returns = np.empty(n_ep)
    actions = np.empty(n_ep)
    
    name = 'Pure exploitation'
    for e in tqdm(range(n_ep)):
        #action = np.argmax(Q)
        action = np.random.randint(len(env.action_space.n))
        N[action]+= 1
        _, reward, _, _ = env.step(action)
        Q[action] += (reward - Q[action])/N[action] 
        #reason for subtraction is because in this env, rewards are 0 or 1. having a constant +ve reward may cause hangups 
        
        Q_s[e] = Q
        returns[e] = reward
        actions[e] = action
    return name, returns, Qe, actions

In [9]:
def epsilon_greedy_strat(env, n_ep = 300, epsilon = 0.01):
    Q = np.zeros((env.action_space.n)) #remember, mab has only one state so Q table will be 1 by no. of actions
    N = np.zeros((env.action_space.n)) #counts the number of times an action was selected(basically the discount factor)
    
    Q_s = np.empty((n_ep, env.action_space.n))
    returns = np.empty(n_ep)
    actions = np.empty(n_ep)
    
    name = 'Pure exploitation'
    for e in tqdm(range(n_ep)):
        #action = np.argmax(Q)
        if np.random.random() > epsilon:
            action = np.random.randint(len(env.action_space.n))
        else:
            action = np.argmax(Q)
            
        N[action]+= 1
        _, reward, _, _ = env.step(action)
        Q[action] += (reward - Q[action])/N[action] 
        #reason for subtraction is because in this env, rewards are 0 or 1. having a constant +ve reward may cause hangups 
        
        Q_s[e] = Q
        returns[e] = reward
        actions[e] = action
    return name, returns, Qe, actions

In [10]:
def epsilon_greedy_strat_fl(env, n_ep = 300, epsilon = 0.001):
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    V = np.zeros((env.observation_space.n)) 
    N = np.zeros((env.observation_space.n, env.action_space.n)) #used to prevent overflow
    gamma = 0.99
   
    for e in (range(n_ep)):
        state = env.reset()
        while True:
            if np.random.random() > epsilon:
                action = np.random.randint((env.action_space.n))
            else:
                action = np.argmax(Q[state])
            
            N[state][action]+= 1
            new_state, reward, done, _ = env.step(action)
            Q[state][action] += (reward - gamma*V[new_state])/N[state][action] 
            #reason for subtraction is because in this env, rewards are 0 or 1. having a constant +ve reward may cause hangups 
            state = new_state
            if done:
                break
        
        V = np.max(Q, axis=1)
    return Q, V

In [11]:
def lin_dec_epsilon_greedy_fl(env,
                          init_epsilon = 1.0,
                          min_epsilon=0.01,
                          decay_ratio = 0.05,
                          n_ep = 4000):
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    V = np.zeros((env.observation_space.n)) 
    N = np.zeros((env.observation_space.n, env.action_space.n)) #used to prevent overflow
    gamma = 0.99
    ep_arr = []
    
    for episode in range(n_ep):
        decay_episodes = n_ep * decay_ratio
        epsilon = 1 - episode/decay_episodes
        epsilon *= init_epsilon - min_epsilon
        epsilon+=min_epsilon
        epsilon = np.clip(epsilon, min_epsilon, init_epsilon)
        total_rewards = 0
        state = env.reset()
        while True:
            if np.random.random() > epsilon:
                action =  np.argmax(Q[state])
            else:
                action = np.random.randint(env.action_space.n)
            
            new_state, reward, done, _ = env.step(action)
            N[state][action]+=1
            Q[state][action]+= (reward + gamma*V[new_state])/N[state][action]
            state = new_state
            total_rewards+=reward
            if done == True:
                break
        V = np.max(Q, axis=1)
        ep_arr.append(total_rewards)
    return V,Q,ep_arr      

In [12]:
def optimistic_initialization(env,
                             optimistic_estimate = 1.0,
                             initial_count = 100,
                             n_ep = 5000):
    Q = np.full((env.observation_space.n,env.action_space.n ),optimistic_estimate)
    N = np.full((env.observation_space.n,env.action_space.n),initial_count)
    V = np.zeros((env.observation_space.n))
    gamma = 0.99
    for e in range(n_ep):
        state = env.reset()
        while True:
            action = np.argmax(Q[state])
            new_state, reward, done, _ = env.step(action)
            N[state][action]+=1
            Q[state][action]+= (reward + gamma*V[new_state])/N[state][action]
            state = new_state
            
            if done == True:
                break
        V = np.max(Q, axis=1)
    return V, Q, N


In [13]:
import math
def softmax(env,
           init_temp = 1000.0,
           min_temp = 0.01,
           decay_ratio = 0.04,
           n_episodes = 5000):
    
    theta = 1e-10
    Q = np.full((env.observation_space.n, env.action_space.n), theta)
    N = np.zeros((env.observation_space.n, env.action_space.n))
    V = np.zeros((env.observation_space.n))
    gamma = 0.99
    
    for e in range(n_episodes):
        decay_episodes = n_episodes*decay_ratio
        temp = 1 - e/decay_episodes
        temp*=init_temp - min_temp
        temp+=min_temp
        temp = np.clip(temp, min_temp, init_temp) #makes sure temp isnt 0
        
        scaled_Q = Q/temp #add temp
        
        norm_Q = scaled_Q - np.max(scaled_Q, axis = 1).reshape(-1, 1)#norm for stability
        exp_Q = np.exp(norm_Q)
        probs = exp_Q/np.sum(exp_Q, axis=1).reshape(-1,1)
        
        if math.isnan(probs[0].sum()) == True:
            print(probs)
        
        assert np.isclose(probs[0].sum(), 1.0)
        
        state = env.reset()
        while True:
            action = np.random.choice(np.arange(env.action_space.n),
                                     size = 1,
                                     p = probs[state])[0]
            new_state, reward, done, _ = env.step(action)
            N[state][action]+=1
            Q[state][action]+= (reward + gamma*V[new_state])/N[state][action]
            state = new_state
            
            if done == True:
                break
        V = np.max(Q, axis = 1)
    return V, Q
            

In [418]:
#V,Q = softmax(env,
#           init_temp = 1000.0,
#           min_temp = 0.01,
#           decay_ratio = 0.05,
#           n_episodes = 100000)

In [390]:
#V,Q,N = optimistic_initialization(env,
#                             optimistic_estimate = 1.0,
#                             initial_count = 100,
#                             n_ep = 100000)

In [393]:
#V,Q,R = lin_dec_epsilon_greedy_fl(env,
#                          init_epsilon = 0.8,
#                          min_epsilon=0.1,
#                          decay_ratio = 0.05,
#                          n_ep = 100000)

In [419]:
#epsilon_greedy_strat_fl(env, n_ep = 150, epsilon = 0.01)
#q_table = Q

In [1]:
#maxStepsPerEpisode = 100
#import time
#from IPython.display import clear_output
#for episode in range(10):
#    state = env.reset()
#    print("EPISODE: ", episode+1)
#    time.sleep(2)

#    for step in range(maxStepsPerEpisode):
#        clear_output(wait=True)
#        env.render()
#        time.sleep(0.5)

#        action = np.argmax(q_table[state, :])
#        new_state, reward, done, info = env.step(action)

#        if done:
#            clear_output(wait=True)
#            env.render()
#            if reward == 1:
#                print("successfull goal")
#                time.sleep(2)
#            elif reward == -1:
#                print("hole")
#                time.sleep(2)
#            clear_output(wait=True)
#            break

#    state = new_state
#env.close()